# reduceByKeyAndWindow transformation Exercise

Spark Streaming also provides windowed computations, which allow you to apply transformations over a sliding window of data. The following figure illustrates this sliding window.


Some of the common window operations are as follows. All of these operations take the said two parameters - windowLength and slideInterval.

| Transformation        | Meaning           |
| -------------:|:-------------|
| **reduceByKeyAndWindow**(func, windowLength, slideInterval, [numTasks])     | When called on a DStream of (K, V) pairs, returns a new DStream of (K, V) pairs where the values for each key are aggregated using the given reduce function func over batches in a sliding window. Note: By default, this uses Spark's default number of parallel tasks (2 for local mode, and in cluster mode the number is determined by the config property spark.default.parallelism) to do the grouping. You can pass an optional numTasks argument to set a different number of tasks. |
| **reduceByKeyAndWindow**(func, invFunc, windowLength, slideInterval, [numTasks])      | A more efficient version of the above reduceByKeyAndWindow() where the reduce value of each window is calculated incrementally using the reduce values of the previous window. This is done by reducing the new data that enters the sliding window, and “inverse reducing” the old data that leaves the window. An example would be that of “adding” and “subtracting” counts of keys as the window slides. However, it is applicable only to “invertible reduce functions”, that is, those reduce functions which have a corresponding “inverse reduce” function (taken as parameter invFunc). Like in reduceByKeyAndWindow, the number of reduce tasks is configurable through an optional argument. Note that checkpointing must be enabled for using this operation.      |

Explain reduceByKeyAndWindow transformation in depth and what is the usage of reduceByKeyAndWindow function

### Exercise

In [1]:
import os
import pathlib
import findspark

os.environ['SPARK_HOME'] = '/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/spark-3.5.0-bin-hadoop3'
os.environ['PYSPARK_DEIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'


findspark.init()
findspark.find()

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark import SparkConf
from apache_log_parser import ApacheAccessLog

sc = SparkContext()
ssc = StreamingContext(sparkContext=sc, batchDuration=2)
ssc.checkpoint('checkpoint')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/16 14:33:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/spark-3.5.0-bin-hadoop3/python/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [2]:
# create DStream from text file
# Note: the spark streaming checks for any updates to this directory.
# So first, start this program, and then copy the log file logs/access_log.log to 'directory' location

curr = pathlib.Path().resolve()
logs_directory = os.path.join(curr / 'logs')
log_data = ssc.textFileStream(logs_directory)
access_log_dstream = log_data.map(ApacheAccessLog.parse_from_log_line).filter(lambda parsed_line: parsed_line is not None)
ip_dstream = access_log_dstream.map(lambda parsed_line: (parsed_line.ip, 1)) 
ip_count = ip_dstream.reduceByKey(lambda x,y: x+y)
# ip_count.pprint(num = 30)
ip_bytes_dstream = access_log_dstream.map(lambda parsed_line: (parsed_line.ip, parsed_line.content_size))
ip_bytes_sum_dstream = ip_bytes_dstream.reduceByKey(lambda x,y: x+y)
ip_bytes_request_count_dstream = ip_count.join(ip_bytes_sum_dstream)
# ip_bytes_request_count_dstream.pprint(num = 30)

In [7]:
####### TODO: use reduceByKeyAndWindow() to get Ip counts per window ###########

ip_dstream.reduceByKeyAndWindow(
    func=lambda a, b: (a + b),
    invFunc= lambda a, b: (a - b),
    windowDuration=20,
    slideDuration=10
).pprint()

####### Exercise End ##########################################################

In [8]:
ssc.start() 
# ssc.awaitTermination()

-------------------------------------------
Time: 2023-11-16 14:41:08
-------------------------------------------



-------------------------------------------
Time: 2023-11-16 14:41:08
-------------------------------------------



-------------------------------------------
Time: 2023-11-16 14:41:18
-------------------------------------------



-------------------------------------------
Time: 2023-11-16 14:41:18
-------------------------------------------



-------------------------------------------
Time: 2023-11-16 14:41:28
-------------------------------------------



...

-------------------------------------------
Time: 2023-11-16 14:42:48
-------------------------------------------



-------------------------------------------
Time: 2023-11-16 14:42:58
-------------------------------------------
('lj1036.inktomisearch.com', 1)
('lj1090.inktomisearch.com', 1)
('64.242.88.10', 50)
('lordgun.org', 1)



-------------------------------------------
Time: 2023-11-16 14:42:58
-------------------------------------------
('lj1036.inktomisearch.com', 1)
('lj1090.inktomisearch.com', 1)
('64.242.88.10', 50)
('lordgun.org', 1)



-------------------------------------------
Time: 2023-11-16 14:43:08
-------------------------------------------
('lj1036.inktomisearch.com', 1)
('lj1090.inktomisearch.com', 1)
('64.242.88.10', 50)
('lordgun.org', 1)



-------------------------------------------
Time: 2023-11-16 14:43:08
-------------------------------------------
('lj1036.inktomisearch.com', 1)
('lj1090.inktomisearch.com', 1)
('64.242.88.10', 50)
('lordgun.org', 1)



In [ ]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

## References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#window-operations